In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
import csv
import numpy as np
import sklearn
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data cleaning

In [ ]:
years = [year for year in range(2021, 2009, -1)]
#years =['2011', '2010']

In [ ]:
import re

def process_excel_file(year, excel_path):
    df = pd.read_excel(excel_path, sheet_name=1, header=None)


    df = df.transpose().reset_index()
    df.columns = df.iloc[0]
    # Drop the first row
    df = df.iloc[1:]

    df.columns = ['Owner_occupied_' + col if 10>= i > 4 else col for i, col in enumerate(df.columns)]
    df.columns = ['Renter_occupied_' + col if 17>= i > 11 else col for i, col in enumerate(df.columns)]
    df = df.rename(columns={'Owner occupied:' : 'Owner_occupied', 'Renter occupied:' : 'Renter_occupied'})

    new_column_name = 'census_tract'  # Replace with the new name
    df.columns.values[1] = new_column_name
    df = df.dropna(subset=['census_tract'])
    #df['census_tract'] = df['census_tract'].apply(lambda x: re.sub(r'[^0-9.]', '', x) if isinstance(x, str) else x)
    df['census_tract'] = df['census_tract'].apply(lambda x: re.sub(r'[^0-9]', '', x) if isinstance(x, str) else x)
    df = df.reset_index()
    df = df.drop(columns = ['Label', 'index', 0], axis=1)
    df['year'] = year




    return df

In [ ]:
for year in years:
  excel_file_path = f'/content/drive/MyDrive/DS project/Roxbury_homeownership/ACSDT5Y{year}.xlsx'
  variable_name = f'Roxbury_homeownership_{year}'

        # Call the function and assign the result to the variable
  globals()[variable_name] = process_excel_file(year, excel_file_path)

In [ ]:
for year in years:
  df = globals()[f'Roxbury_homeownership_{year}']
  for column in df.columns[: -1]:
    df[column] = df[column].str.replace('[^0-9-]', '', regex=True).astype(int)

In [ ]:
for df in [Roxbury_homeownership_2021, Roxbury_homeownership_2020, Roxbury_homeownership_2019, Roxbury_homeownership_2018]:

  df['Owner_occupied_Moved in 1999 or earlier'] =  df.iloc[:, 7:9].sum(axis=1)
  df['Renter_occupied_Moved in 1999 or earlier'] = df.iloc[:, 14:16].sum(axis=1)

In [ ]:
for df in [Roxbury_homeownership_2017, Roxbury_homeownership_2016, Roxbury_homeownership_2015]:
  df['Owner_occupied_Moved in 1999 or earlier'] =  df.iloc[:, 6:9].sum(axis=1)
  df['Renter_occupied_Moved in 1999 or earlier'] = df.iloc[:, 13:16].sum(axis=1)

In [ ]:
for df in [Roxbury_homeownership_2014, Roxbury_homeownership_2013, Roxbury_homeownership_2012, Roxbury_homeownership_2011, Roxbury_homeownership_2010]:
  df['Owner_occupied_Moved in 1999 or earlier'] =  df.iloc[:, 5:9].sum(axis=1)
  df['Renter_occupied_Moved in 1999 or earlier'] = df.iloc[:, 12:16].sum(axis=1)

In [ ]:

Roxbury_homeownership_2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 19 columns):
 #   Column                                       Non-Null Count  Dtype
---  ------                                       --------------  -----
 0   census_tract                                 18 non-null     int64
 1   Total population in occupied housing units:  18 non-null     int64
 2   Owner_occupied                               18 non-null     int64
 3   Owner_occupied_Moved in 2010 or later        18 non-null     int64
 4   Owner_occupied_Moved in 2000 to 2009         18 non-null     int64
 5   Owner_occupied_Moved in 1990 to 1999         18 non-null     int64
 6   Owner_occupied_Moved in 1980 to 1989         18 non-null     int64
 7   Owner_occupied_Moved in 1970 to 1979         18 non-null     int64
 8   Owner_occupied_Moved in 1969 or earlier      18 non-null     int64
 9   Renter_occupied                              18 non-null     int64
 10  Renter_occupied_Moved in 201

In [ ]:
#list_of_indices
# Create a list of DataFrames
concatenated_df = [globals()[f'Roxbury_homeownership_{year}'] for year in years]

# Initialize an empty DataFrame to store the concatenated result
result_df2 = pd.DataFrame()

# Concatenate the DataFrames in the list
for df in concatenated_df:
    result_df2 = pd.concat([result_df2, df], ignore_index=True)

In [ ]:
result_df2.columns

Index(['census_tract', 'Total population in occupied housing units:',
       'Owner_occupied', 'Owner_occupied_Moved in 2019 or later',
       'Owner_occupied_Moved in 2015 to 2018',
       'Owner_occupied_Moved in 2010 to 2014',
       'Owner_occupied_Moved in 2000 to 2009',
       'Owner_occupied_Moved in 1990 to 1999',
       'Owner_occupied_Moved in 1989 or earlier', 'Renter_occupied',
       'Renter_occupied_Moved in 2019 or later',
       'Renter_occupied_Moved in 2015 to 2018',
       'Renter_occupied_Moved in 2010 to 2014',
       'Renter_occupied_Moved in 2000 to 2009',
       'Renter_occupied_Moved in 1990 to 1999',
       'Renter_occupied_Moved in 1989 or earlier', 'year',
       'Owner_occupied_Moved in 1999 or earlier',
       'Renter_occupied_Moved in 1999 or earlier',
       'Owner_occupied_Moved in 2017 or later',
       'Owner_occupied_Moved in 2015 to 2016',
       'Renter_occupied_Moved in 2017 or later',
       'Renter_occupied_Moved in 2015 to 2016',
       'Owner_

In [ ]:
result_df3 = result_df2[['census_tract', 'Total population in occupied housing units:',
       'Owner_occupied', 'Owner_occupied_Moved in 1999 or earlier',
      'Renter_occupied', 'Renter_occupied_Moved in 1999 or earlier',
       'year']]

In [ ]:
result_df3

,census_tract,Total population in occupied housing units:,Owner_occupied,Owner_occupied_Moved in 1999 or earlier,Renter_occupied,Renter_occupied_Moved in 1999 or earlier,year
0,70801,1096,23,0,1073,22,2021
1,70901,1141,282,2,859,26,2021
2,801,1658,443,249,1215,141,2021
3,803,2371,516,75,1855,170,2021
4,80401,2728,122,0,2606,285,2021
...,...,...,...,...,...,...,...
209,820,2542,954,667,1588,399,2010
210,821,4822,719,467,4103,701,2010
211,904,3529,1220,623,2309,737,2010
212,906,1805,474,430,1331,560,2010


In [ ]:
result_df3.sort_values(by=['census_tract', 'year'], inplace=True)

result_df3['percent_owners'] = result_df3['Owner_occupied']/result_df3['Total population in occupied housing units:']
result_df3['percent_renters'] = result_df3['Renter_occupied']/result_df3['Total population in occupied housing units:']

result_df3['percent_old_owners'] = result_df3['Owner_occupied_Moved in 1999 or earlier']/result_df3['Owner_occupied']
result_df3['percent_old_renters'] = result_df3['Owner_occupied_Moved in 1999 or earlier']/result_df3['Renter_occupied']

result_df3['renters_owners_ratio'] = result_df3['Owner_occupied']/result_df3['Renter_occupied']
result_df3['old_renters_owners_ratio'] = result_df3['Owner_occupied_Moved in 1999 or earlier']/result_df3['Renter_occupied_Moved in 1999 or earlier']

# Calculate the percentage change within each 'census_tract' group
#result_df3['percentage_change'] = result_df3.groupby('census_tract')['Owner_occupied_Moved in 1999 or earlier'].pct_change()

<ipython-input-15-236f53da9bbd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df3.sort_values(by=['census_tract', 'year'], inplace=True)
<ipython-input-15-236f53da9bbd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df3['percent_owners'] = result_df3['Owner_occupied']/result_df3['Total population in occupied housing units:']
<ipython-input-15-236f53da9bbd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [ ]:
result_df3['census_tract'] = result_df3['census_tract'].astype('float64')

# Adding back the decimal to census tract..
def custom_replace(value):
    if value > 10000:
        return value/100
   # else: return int(value)

    return value


result_df3['census_tract'] = result_df3['census_tract'].apply(custom_replace)

<ipython-input-16-1fae1e3157fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df3['census_tract'] = result_df3['census_tract'].astype('float64')
<ipython-input-16-1fae1e3157fa>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df3['census_tract'] = result_df3['census_tract'].apply(custom_replace)


In [ ]:
result_df3.reset_index(drop=True)

,census_tract,Total population in occupied housing units:,Owner_occupied,Owner_occupied_Moved in 1999 or earlier,Renter_occupied,Renter_occupied_Moved in 1999 or earlier,year,percent_owners,percent_renters,percent_old_owners,percent_old_renters,renters_owners_ratio,old_renters_owners_ratio
0,708.00,3436,959,378,2477,355,2010,0.279104,0.720896,0.394161,0.152604,0.387162,1.064789
1,708.00,3531,996,273,2535,305,2011,0.282073,0.717927,0.274096,0.107692,0.392899,0.895082
2,708.00,3737,955,251,2782,470,2012,0.255553,0.744447,0.262827,0.090223,0.343278,0.534043
3,708.00,3947,1127,257,2820,497,2013,0.285533,0.714467,0.228039,0.091135,0.399645,0.517103
4,708.00,3643,1318,417,2325,373,2014,0.361790,0.638210,0.316388,0.179355,0.566882,1.117962
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,806.01,1865,312,21,1553,403,2019,0.167292,0.832708,0.067308,0.013522,0.200901,0.052109
210,806.01,1898,324,28,1574,382,2020,0.170706,0.829294,0.086420,0.017789,0.205845,0.073298
211,806.01,1807,355,28,1452,210,2021,0.196458,0.803542,0.078873,0.019284,0.244490,0.133333
212,813.02,2210,114,0,2096,158,2020,0.051584,0.948416,0.000000,0.000000,0.054389,0.000000


In [ ]:
Roxbury_houseownership = result_df3[result_df3['census_tract'] != 9803]

In [ ]:
Roxbury_houseownership.to_csv('/content/drive/MyDrive/DS project/Roxbury_homeownership/Roxbury_houseownership_without9803.csv')